In [7]:
class BS_pricer():
    @staticmethod
    def BlackScholes(payoff='call', S0=100., K=100., T=1., r=0.1, sigma=0.2 ):  
        d1 = (np.log(S0/K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = (np.log(S0/K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))

        if payoff=="call":
            return S0 * ss.norm.cdf( d1 ) - K * np.exp(-r * T) * ss.norm.cdf( d2 )
        elif payoff=="put":
            return K * np.exp(-r * T) * ss.norm.cdf( -d2 ) - S0 * ss.norm.cdf( -d1 )
        else:
            raise ValueError("invalid type. Set 'call' or 'put'")

In [8]:
import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from datetime import date

In [13]:
now = datetime.today().strftime('%Y:%m:%d')
d1 = datetime.strptime(now, '%Y:%m:%d')
stk = yf.Ticker("MSFT")
expiry = stk.options[6]         #expiry date choice
opt = stk.option_chain(expiry)  #selecting that option based on expiry date                         
low = opt.calls.iloc[10]          #selecting specific call
low1 = opt.puts.iloc[10]          #selecting specific put
d2 = datetime.strptime(expiry, '%Y-%m-%d')
diff = (d2 - d1).days
print(opt.puts)

         contractSymbol       lastTradeDate  strike  lastPrice  bid  ask  \
0   MSFT200828P00180000 2020-07-14 18:58:28   180.0       2.79  0.0  0.0   
1   MSFT200828P00185000 2020-07-14 19:39:57   185.0       3.54  0.0  0.0   
2   MSFT200828P00187500 2020-07-14 19:27:09   187.5       4.35  0.0  0.0   
3   MSFT200828P00190000 2020-07-14 19:18:31   190.0       4.75  0.0  0.0   
4   MSFT200828P00192500 2020-07-14 18:18:21   192.5       5.85  0.0  0.0   
5   MSFT200828P00195000 2020-07-14 19:46:22   195.0       6.35  0.0  0.0   
6   MSFT200828P00197500 2020-07-14 19:49:18   197.5       7.40  0.0  0.0   
7   MSFT200828P00200000 2020-07-14 18:54:19   200.0       8.55  0.0  0.0   
8   MSFT200828P00202500 2020-07-14 19:45:54   202.5       9.36  0.0  0.0   
9   MSFT200828P00205000 2020-07-14 19:39:57   205.0      10.57  0.0  0.0   
10  MSFT200828P00207500 2020-07-14 19:46:22   207.5      11.55  0.0  0.0   
11  MSFT200828P00210000 2020-07-14 17:47:39   210.0      13.26  0.0  0.0   
12  MSFT2008

In [10]:
S0 = stk.info['open']
K= low.loc['strike']    # strike
T= diff/365      # maturity 
r= 0.011       # risk free rate 
sig = low.loc['impliedVolatility'] 
call = BS_pricer.BlackScholes("call",S0,K,T,r,sig) 

In [11]:
S0 = stk.info['open']
K= low1.loc['strike']    # strike
T= diff/365      # maturity 
r= 0.011       # risk free rate 
sig = low1.loc['impliedVolatility'] 
put = BS_pricer.BlackScholes("put",S0,K,T,r,sig) 
print("Call price: ", call )  
print("Put price: ", put ) 

Call price:  0.0
Put price:  0.0556014373362288


In [12]:
print("Difference between model and actual (call): ", (low.loc['lastPrice'] - call))
print("Difference between model and actual (put): ", (low1.loc['lastPrice'] - put))

Difference between model and actual (call):  11.6
Difference between model and actual (put):  10.514398562663771
